In [2]:
#Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from PIL import Image
import clip
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt



In [ ]:
torch.cuda.is_available()

True

In [3]:
data = pd.read_csv('cat_dog1.csv')
data

,Unnamed: 0,image,labels
0,0,dog.624.jpg,1
1,1,cat.170.jpg,0
2,2,dog.419.jpg,1
3,3,dog.19.jpg,1
4,4,cat.227.jpg,0
...,...,...,...
1995,1995,dog.481.jpg,1
1996,1996,dog.159.jpg,1
1997,1997,cat.576.jpg,0
1998,1998,dog.190.jpg,1


In [4]:
#a-Spliting the dataset into training and testing sets with 20% for test dataset
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [5]:
train_data.sort_values(train_data.columns[2], axis = 0, inplace=True) #sort by label
train_cats = train_data[train_data['labels'] == 0]
train_dogs = train_data[train_data['labels'] == 1]


In [6]:
# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"

model, transform = clip.load("ViT-B/32", device) 

In [7]:
preproc_img_train_cat = []
preproc_img_train_dog = []
preproc_img_test = []

In [8]:
for file in train_cats['image']:
  preproc_img_train_cat.append(transform(Image.open('cat_dog1/' + file )).unsqueeze(0).to(device))
for file in train_dogs['image']:
  preproc_img_train_dog.append(transform(Image.open('cat_dog1/' + file )).unsqueeze(0).to(device))

for file in test_data['image']:
  preproc_img_test.append(transform(Image.open('cat_dog1/' + file)).unsqueeze(0).to(device))

In [10]:
#encode the inputs
image_embeddings_train_dog = []
image_embeddings_train_cat = []
image_embeddings_test = []
torch.cuda.empty_cache()
with torch.no_grad():
  for image in preproc_img_train_cat:
    image_embeddings_train_cat.append((model.encode_image(image)).cpu().detach().numpy())
  
  for image in preproc_img_train_dog:
    image_embeddings_train_dog.append((model.encode_image(image)).cpu().detach().numpy())

  for image in preproc_img_test:
    image_embeddings_test.append((model.encode_image(image)).cpu().detach().numpy())


In [11]:
image_embeddings_train_cat = np.vstack(image_embeddings_train_cat)
image_embeddings_train_dog = np.vstack(image_embeddings_train_dog)
image_embeddings_test = np.vstack(image_embeddings_test)


In [25]:
#Fisher's Algorithm
c = 0.1
means = []
means.append(image_embeddings_train_cat.mean(axis = 0))
means.append(image_embeddings_train_dog.mean(axis = 0))
S =  np.cov(image_embeddings_train_cat, rowvar=False) + np.cov(image_embeddings_train_dog, rowvar=False)
S_inv = np.linalg.inv(S)
mean_dif = means[1] - means[0]
w = c*np.dot(S_inv, mean_dif)

In [26]:
#predictions with c = 0.1
predict_c = np.dot(image_embeddings_test, w.T)

In [30]:
#Scores
#Convert prediction scores into binary class predictions using a 0.0 threshold
predicted_labels = (predict_c > 0.0).astype(int)

#Calculations of metrics
accuracy = accuracy_score(test_data['labels'], predicted_labels)
precision = precision_score(test_data['labels'], predicted_labels)
recall = recall_score(test_data['labels'], predicted_labels)
f1 = f1_score(test_data['labels'], predicted_labels)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

#Generating Confusion Matrix
confusion_matrix = confusion_matrix(test_data['labels'], predicted_labels)
sns.heatmap(confusion_matrix, annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.ylabel('Actual labels')
plt.xlabel('Predicted labels')
plt.show()

Accuracy: 0.9825
Precision: 1.0
Recall: 0.964824120603015
F1 Score: 0.9820971867007673


TypeError: 'numpy.ndarray' object is not callable

In [15]:
#Bonus

C = [0.2, 0.3, 0.4, 0.5, 0.7]
W = []
for c in C:
  W.append(c*np.dot(S_inv, mean_dif))



In [16]:
predict_C = []
for w in W:
  predict_C.append(np.dot(image_embeddings_test, w.T))

In [20]:
#Scores
#Loop through each of the c values to calculate and print metrics for each
for i, c in enumerate(C):
    predicted_labelsC = (predict_C[i] > 0.0).astype(int)
    
    #Calculations of metrics
    accuracyB = accuracy_score(test_data['labels'], predicted_labelsC)
    precisionB = precision_score(test_data['labels'], predicted_labelsC)
    recallB = recall_score(test_data['labels'], predicted_labelsC)
    f1B = f1_score(test_data['labels'], predicted_labelsC)
    
    print(f'Metrics for c={c}:')
    print('Accuracy:', accuracyB)
    print('Precision:', precisionB)
    print('Recall:', recallB)
    print('F1 Score:', f1B)
    
#Generating Confusion Matrix
    confusion_matrix = confusion_matrix(test_data['labels'], predicted_labelsC)
    sns.heatmap(confusion_matrix, annot=True, fmt='d')
    plt.title(f'Confusion Matrix for c={c}')
    plt.ylabel('Actual labels')
    plt.xlabel('Predicted labels')
    plt.show()


Metrics for c=0.2:
Accuracy: 0.9825
Precision: 1.0
Recall: 0.964824120603015
F1 Score: 0.9820971867007673


TypeError: 'numpy.ndarray' object is not callable